In [1]:
import pandas as pd
import numpy as np

from textblob import TextBlob
from nltk.tokenize import WordPunctTokenizer, word_tokenize, StanfordSegmenter, sent_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords

import re, os, sys, string, itertools
from collections import defaultdict, Counter
import unicodedata
from tqdm import tqdm, tqdm_notebook

In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [4]:
def translate_quoted_words(series):
    """ This functions replaces words which are written in quotes
     with quoted word e.g. "Damn" with "quoted "Damn""
     """
    reg = re.compile(r'((?<=")"[\w\s]+"(?="))')
    print("Total occurences of quoted words {}".format(series.str.count(reg).sum()))
    series = series.str.replace(reg, 'quoted \\1')
    return series

In [5]:
train.comment_text = translate_quoted_words(train.comment_text)


Total occurences of quoted words 57030


In [ ]:
def break_oovwords(series, vocab_filename, sp_file):
    dict_words = set([o.rstrip().rsplit(' ')[0] for o in open(vocab_filename)])
    sp = spm.SentencePieceProcessor()
    sp.Load(sp_file)
    

In [6]:
text = train.comment_text.iloc[140]
text = "fuckin"
print(text)
TextBlob(text).correct()

fuckin


TextBlob("fucking")

In [7]:
embed_file ="../utility/glove.42B.300d.txt"
glove_words = [o.rstrip().rsplit(' ')[0] for o in open(embed_file)]

In [8]:
len(glove_words)

1917494

In [9]:
glove_set = set(glove_words)

In [10]:
cnt = Counter(itertools.chain.from_iterable(train.comment_text.str.lower().replace("'","").replace("\n{1,}", " ").str.replace("[^a-zA-Z\s]+", " ").str.split()))

In [12]:
words = set(cnt.keys())

In [13]:
oov_words = words - glove_set
len(oov_words)

47616

In [34]:
fk_oov_words = [word for word in oov_words if "uck" in word]

In [35]:
print(len(fk_oov_words))
fk_oov_words

207


['luckrfa',
 'marcolfuck',
 'suckybaby',
 'swagfuckingtastic',
 'touchmyshitagainandyoulllearnwahtitmeanstotossasaladfatnerdfuckingtrollpieceofshitasswipe',
 'cksucking',
 'fuckyourselves',
 'cacknuck',
 'fuckchop',
 'cansuck',
 'chucknorrisfacts',
 'thatsphucked',
 'bucksham',
 'motehrfucker',
 'puckerd',
 'dickfucker',
 'muckanaghederdauhaulia',
 'qucky',
 'rfucker',
 'suckersyou',
 'fuckknuckle',
 'fuckingmom',
 'tuckdogg',
 'suckitpedia',
 'muckamucks',
 'niggercocksucking',
 'canuckster',
 'dicksuckington',
 'ruckess',
 'craphellinabucket',
 'sucksvvv',
 'suckpuppet',
 'djsuckafree',
 'lunarmoonbatssuck',
 'shmucky',
 'suckpoppetry',
 'shitfucker',
 'bubbachuck',
 'securityfuck',
 'fuckboys',
 'guckelberger',
 'duckecho',
 'cardinaldanfuck',
 'sucket',
 'nuckelavee',
 'niggafuck',
 'smuckola',
 'fuckcock',
 'discofucker',
 'motherucker',
 'bamafuck',
 'ducktaped',
 'suckipedia',
 'justuck',
 'suckpoper',
 'saltchuck',
 'fuckum',
 'fuckyouself',
 'michaelsavagesucks',
 'szczucka',


In [131]:
%load_ext cython

In [49]:
print(train.comment_text.str.lower().iloc[161])
' '.join(itertools.chain.from_iterable([sp.EncodeAsPieces(word) if word not in glove_set else [word] for word in train.comment_text.str.lower().iloc[161].split(' ') ]))

"

actually, direct quotes that aren't in quotation marks and aren't attributed are generally known as "quoted "plagiarism"" and/or ""copyright infringement."" but the nyt material isn't the issue, as you well know. the material from the "quoted "jewish action taskforce"" is the problem.  "


'▁" \n\n actually , direct quotes that ▁aren \' t in quotation marks and ▁aren \' t attributed are generally known as ▁" quoted ▁" plag iar ism "" and/or ▁"" copyright ▁infringement ."" but the nyt material ▁isn \' t the ▁issue , as you well know. the material from the ▁" quoted ▁" jewish action ▁taskforce "" is the problem. "'

In [22]:
[TextBlob(word).correct() for word in tqdm(fk_oov_words) if len(word) < 15]

100%|██████████| 88/88 [00:25<00:00,  3.51it/s]


[TextBlob("marcolfuck"),
 TextBlob("fuckyourselves"),
 TextBlob("fuckchop"),
 TextBlob("motehrfucker"),
 TextBlob("dickfucker"),
 TextBlob("pucker"),
 TextBlob("fuckknuckle"),
 TextBlob("fuckingmom"),
 TextBlob("shitfucker"),
 TextBlob("securityfuck"),
 TextBlob("fuckboys"),
 TextBlob("niggafuck"),
 TextBlob("fuckcock"),
 TextBlob("discofucker"),
 TextBlob("bamafuck"),
 TextBlob("fuck"),
 TextBlob("fuckyouself"),
 TextBlob("fuckelwad"),
 TextBlob("fuckwqit"),
 TextBlob("fuckpost"),
 TextBlob("pinkafuckup"),
 TextBlob("fuckersitalic"),
 TextBlob("yingfuck"),
 TextBlob("denverfuckedme"),
 TextBlob("animalfucker"),
 TextBlob("fuckwitz"),
 TextBlob("funniest"),
 TextBlob("sisterfucker"),
 TextBlob("fuckturd"),
 TextBlob("textfuck"),
 TextBlob("gofuckyourself"),
 TextBlob("penisfuck"),
 TextBlob("fuckingabf"),
 TextBlob("deadlock"),
 TextBlob("fuckbold"),
 TextBlob("defect"),
 TextBlob("cousinfuckers"),
 TextBlob("bitchfuck"),
 TextBlob("muthfucker"),
 TextBlob("everyfucking"),
 TextBlob("m

In [51]:
'isnt' in glove_set

True

In [32]:
%%time
#from autocorrect import spell
TextBlob("uck").correct()

CPU times: user 581 µs, sys: 0 ns, total: 581 µs
Wall time: 587 µs


TextBlob("luck")

In [24]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load("../utility/en.wiki.bpe.op200000.model")


True

In [29]:
[sp.EncodeAsPieces(word) for word in fk_oov_words]

[['▁marc', 'olf', 'uck'],
 ['▁swag', 'fucking', 't', 'astic'],
 ['▁touch',
  'my',
  'shit',
  'again',
  'and',
  'yo',
  'ull',
  'learn',
  'wah',
  'tit',
  'mean',
  'st',
  'ot',
  'oss',
  'asal',
  'ad',
  'fat',
  'ner',
  'df',
  'ucking',
  'troll',
  'piece',
  'of',
  'shit',
  'ass',
  'wipe'],
 ['▁f', 'ucky', 'ours', 'elves'],
 ['▁fuck', 'chop'],
 ['▁mote', 'hr', 'fucker'],
 ['▁dick', 'fucker'],
 ['▁rf', 'ucker'],
 ['▁fuck', 'knuckle'],
 ['▁fucking', 'mom'],
 ['▁shit', 'fucker'],
 ['▁security', 'fuck'],
 ['▁fuck', 'boys'],
 ['▁cardinal', 'dan', 'fuck'],
 ['▁nig', 'gaf', 'uck'],
 ['▁fuck', 'cock'],
 ['▁disco', 'fucker'],
 ['▁bam', 'af', 'uck'],
 ['▁fuck', 'um'],
 ['▁f', 'ucky', 'ous', 'elf'],
 ['▁fuck', 'el', 'wad'],
 ['▁fuck', 'w', 'q', 'it'],
 ['▁fuck', 'post'],
 ['▁pink', 'af', 'uck', 'up'],
 ['▁fuck', 'ers', 'italic'],
 ['▁ying', 'fuck'],
 ['▁denver', 'fuck', 'ed', 'me'],
 ['▁animal', 'fucker'],
 ['▁xxxx', 'xx', 'mother', 'fucker'],
 ['▁fuck', 'witz'],
 ['▁fuck', 'ies

In [33]:
"tit" in glove_set

True

In [31]:
import enchant
d = enchant.Dict("en_US")
d.suggest("uck")

['ck',
 'suck',
 'ruck',
 'tuck',
 'luck',
 'duck',
 'muck',
 'puck',
 'buck',
 'yuck',
 'Tuck',
 'Puck',
 'Huck',
 'Buck',
 'u ck']

In [192]:
%%time
d.suggest("aircracft")

CPU times: user 68.2 ms, sys: 0 ns, total: 68.2 ms
Wall time: 67.4 ms


['aircraft', 'airlift', 'aggregative', 'aggregate']

In [ ]:
#
oov_words = set(oov_words)
oov_words_mod = [d.suggest(word) for word in tqdm(oov_words)]



  0%|          | 0/47616 [00:00<?, ?it/s]

  0%|          | 2/47616 [00:00<40:38, 19.53it/s]

  0%|          | 5/47616 [00:00<48:58, 16.20it/s]

  0%|          | 8/47616 [00:00<46:24, 17.09it/s]

  0%|          | 10/47616 [00:00<47:50, 16.58it/s]

  0%|          | 13/47616 [00:00<45:17, 17.52it/s]

  0%|          | 16/47616 [00:00<44:15, 17.92it/s]

  0%|          | 19/47616 [00:01<42:54, 18.49it/s]

  0%|          | 22/47616 [00:01<41:53, 18.94it/s]

  0%|          | 26/47616 [00:01<39:40, 19.99it/s]

  0%|          | 29/47616 [00:01<39:06, 20.28it/s]

  0%|          | 33/47616 [00:01<37:16, 21.28it/s]

  0%|          | 36/47616 [00:01<37:18, 21.25it/s]

  0%|          | 39/47616 [00:01<37:42, 21.03it/s]

  0%|          | 42/47616 [00:02<37:59, 20.87it/s]

  0%|          | 45/47616 [00:02<37:38, 21.06it/s]

  0%|          | 48/47616 [00:02<37:24, 21.19it/s]

  0%|          | 51/47616 [00:02<37:53, 20.93it/s]

  0%|          | 54/47616 [00:02<37:44, 21.00it/s]

  0%|          | 57/47

In [ ]:
oo

In [170]:
oov_words2 = [word for word in tqdm(cnt.keys())  if word not in glove_set]

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001
53001
54001
55001
56001
57001
58001
59001
60001
61001
62001
63001
64001
65001
66001
67001
68001
69001
70001
71001
72001
73001
74001
75001
76001
77001
78001
79001
80001
81001
82001
83001
84001
85001
86001
87001
88001
89001
90001
91001
92001
93001
94001
95001
96001
97001
98001
99001
100001
101001
102001
103001
104001
105001
106001
107001
108001
109001
110001
111001
112001
113001
114001
115001
116001
117001
118001
119001
120001
121001
122001
123001
124001
125001
126001
127001
128001
129001
130001
131001
132001
133001
134001
135001
136001
137001
138001
139001
140001
141001
142001
143001
144001
145001
146001
147001
148001
149001
150001
151001
152001
153001
154001
155001
156001
157001
158001


In [171]:
len(oov_list)

168816